In [ ]:
# ===============================
# 📦 INSTALL DEPENDENCIES
# ===============================
!apt-get install -y wkhtmltopdf
!pip install -q streamlit transformers torch praw snscrape textblob matplotlib seaborn wordcloud weasyprint pdfkit beautifulsoup4 requests pyngrok


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy
  libavahi-core7 libavahi-glib1 libdaemon0 libevdev2 libgudev-1.0-0 libhyphen0
  libinput-bin libinput10 libjson-glib-1.0-0 libjson-glib-1.0-common
  libmbim-glib4 libmbim-proxy libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200
  libnotify4 libnss-mdns libproxy1v5 libqmi-glib5 libqmi-proxy libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5 libqt5printsupport5
  libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libsoup2.4-1
  libsoup2.4-common libudev1 libwacom-bin libwacom-common libwacom9 libwoff1
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1
  libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 

In [ ]:
# ===============================
# 🧠 WRITE Simplified Streamlit App
# ===============================
%%writefile app.py
import streamlit as st
import pandas as pd
import re
import requests
import praw
import random
import snscrape.modules.twitter as sntwitter
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from transformers import pipeline
from weasyprint import HTML

# ------------------------------
# 🔍 Models
# ------------------------------
sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
emotion_model = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion")

# Placeholder MBTI logic
def classify_mbti(posts):
    # For now, simulate MBTI classification (replace with actual model logic)
    mbti_types = ["INTJ", "ENTP", "INFJ", "ISFP", "ESTP", "ISTJ", "ENFP", "ESFJ"]
    return random.choice(mbti_types), random.uniform(0.7, 1.0)

def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    return text.lower()[:512]

def analyze(text):
    if not text.strip():
        return "Neutral", 0.5, "Neutral", 0.0
    sentiment_result = sentiment_model(text[:512])[0]
    emotion_result = emotion_model(text[:512])[0]

    # Determine risk level
    high_risk_emotions = ['anger', 'fear', 'sadness']
    risk_level = "High" if emotion_result['label'] in high_risk_emotions and sentiment_result['label'] == "NEGATIVE" else "Low"
    return sentiment_result['label'], sentiment_result['score'], emotion_result['label'], emotion_result['score'], risk_level

def calculate_mental_state(df, mbti_type):
    # Aggregate sentiment and emotion results to calculate mental state
    negative_count = df[df['Sentiment'] == "NEGATIVE"].shape[0]
    high_risk_emotions = ['anger', 'fear', 'sadness']
    high_risk_count = df[df['Emotion'].isin(high_risk_emotions)].shape[0]

    if negative_count > len(df) * 0.5 and high_risk_count > len(df) * 0.5:
        overall_state = "Critical"
    elif negative_count > len(df) * 0.3 or high_risk_count > len(df) * 0.3:
        overall_state = "At Risk"
    else:
        overall_state = "Stable"

    # Include MBTI insights (placeholder logic, can be expanded)
    if mbti_type in ["INTJ", "INFJ", "ISTJ"]:
        overall_state += " (Analytical Personality)"
    elif mbti_type in ["ENFP", "ESFJ", "ENTP"]:
        overall_state += " (Expressive Personality)"

    return overall_state

def fetch_twitter_posts(username, limit=30):
    try:
        return [tweet.content for i, tweet in enumerate(sntwitter.TwitterUserScraper(username).get_items()) if i < limit]
    except Exception as e:
        return [f"Twitter error: {e}"]

def fetch_reddit_posts(username, limit=30):
    try:
        reddit = praw.Reddit(
            client_id="BZNBfoQviM8oJ14_Ie8Oag",
            client_secret="1GBs2FcrMMF7BPUXTV-twOsR9GRB2g",
            user_agent="MentalHealthAnalyzer"
        )
        user = reddit.redditor(username)
        return [submission.title + " " + submission.selftext for submission in user.submissions.new(limit=limit)]
    except Exception as e:
        return [f"Reddit error: {e}"]

def generate_report(df):
    df.to_csv("report.csv", index=False)

    plt.figure(figsize=(8,5))
    sns.countplot(x=df['Sentiment'], palette="coolwarm")
    plt.title("Sentiment Distribution")
    plt.savefig("sentiment.png")

    plt.figure(figsize=(8,5))
    sns.countplot(x=df['Emotion'], palette="Set3")
    plt.title("Emotion Distribution")
    plt.savefig("emotion.png")

    wordcloud = WordCloud(width=800, height=400).generate(' '.join(df['Post'].tolist()))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.savefig("wordcloud.png")

    # Build HTML safely
    table_rows = "".join(
        f"<tr><td>{row.Post[:100]}...</td><td>{row.Sentiment}</td><td>{row.Confidence:.2f}</td>"
        f"<td>{row.Emotion}</td><td>{row.Emotion_Confidence:.2f}</td></tr>"
        for _, row in df.iterrows()
    )

    html_content = f'''
    <html><head><title>Report</title></head><body>
    <h1>Mental Health & Emotion Analysis</h1>
    <p>Total Posts Analyzed: {len(df)}</p>
    <h2>Charts</h2>
    <img src="sentiment.png" width="600"><br>
    <img src="emotion.png" width="600"><br>
    <img src="wordcloud.png" width="600"><br>
    <h2>Analysis Table</h2>
    <table border="1" cellpadding="5">
    <tr><th>Post</th><th>Sentiment</th><th>Confidence</th><th>Emotion</th><th>Emotion Confidence</th></tr>
    {table_rows}
    </table></body></html>
    '''

    with open("report.html", "w") as f:
        f.write(html_content)

    HTML("report.html").write_pdf("mental_health_report.pdf")

# ------------------------------
# 🌐 Streamlit Frontend
# ------------------------------
st.set_page_config(page_title="Mental Health Analyzer", layout="wide")
st.title("🧠 Social Media Mental Health Analyzer")

platform = st.radio("Select Platform", ["Twitter", "Reddit"])
username = st.text_input("Enter Username")

if st.button("Analyze"):
    with st.spinner("Analyzing..."):
        posts = fetch_twitter_posts(username) if platform == "Twitter" else fetch_reddit_posts(username)
        results = []
        for post in posts:
            clean = preprocess_text(post)
            s, sc, e, ec, risk = analyze(clean)
            results.append((post, s, sc, e, ec, risk))

        df = pd.DataFrame(results, columns=["Post", "Sentiment", "Confidence", "Emotion", "Emotion_Confidence", "Risk Level"])
        # Calculate MBTI personality
        mbti_type, mbti_confidence = classify_mbti(posts)

        # Calculate overall mental state
        mental_state = calculate_mental_state(df, mbti_type)

        st.success("Analysis complete!")
        st.dataframe(df)

        st.subheader("📊 Emotion Distribution")
        emo_counts = df['Emotion'].value_counts()
        st.bar_chart(emo_counts)

        st.subheader("🧠 MBTI Personality Type")
        st.write(f"**Type:** {mbti_type} (Confidence: {mbti_confidence:.2f})")
        st.markdown(f"Learn more about your type [here](https://www.16personalities.com/{mbti_type.lower()}-personality)")

        st.subheader("💡 Overall Mental State")
        st.write(f"**State:** {mental_state}")


        generate_report(df)
        with open("mental_health_report.pdf", "rb") as f:
            st.download_button("📥 Download PDF Report", f, "mental_health_report.pdf", mime="application/pdf")

        st.subheader("🆘 Get Help")
        st.markdown("""
        - [MentalHealth.gov](https://www.mentalhealth.gov/)
        - [BetterHelp](https://www.betterhelp.com/)
        - [7 Cups](https://www.7cups.com/)
        - [National Suicide Prevention Lifeline](https://988lifeline.org/)
        """)


        st.subheader("💬 Chatbot Assistant (Coming Soon)")
        st.markdown("An AI-based assistant will be available here to discuss results and suggest coping strategies.")


: 

In [ ]:
# ===============================
# 🚀 LAUNCH STREAMLIT
# ===============================
from pyngrok import ngrok
import os
import subprocess

# Set your ngrok auth token (do this once per environment)
ngrok.set_auth_token("2wUXXu47xrn6imR2bqk7o7nYA1v_2JepFad7KEtZ7AH68cHc2")

# Kill existing tunnels (optional cleanup)
ngrok.kill()

# Start a tunnel to the streamlit default port 8501
public_url = ngrok.connect(8501)
print("Streamlit app running on:", public_url)

# Launch the Streamlit app in a subprocess
subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501", "--server.enableCORS=false"])


Streamlit app running on: NgrokTunnel: "https://9948-34-19-101-12.ngrok-free.app" -> "http://localhost:8501"


<Popen: returncode: None args: ['streamlit', 'run', 'app.py', '--server.port...>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# ------------------------------
# 🤖 Chatbot Setup
# ------------------------------
@st.cache_resource
def load_chatbot():
    tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
    model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
    return tokenizer, model

tokenizer, model = load_chatbot()

# Keep conversation history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

st.subheader("💬 Chatbot Assistant")
user_input = st.text_input("You: ", key="chat_input")

if st.button("Send"):
    if user_input:
        st.session_state.chat_history.append(f"You: {user_input}")

        # Encode input and generate response
        new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
        bot_input_ids = torch.cat([tokenizer.encode(" ".join(st.session_state.chat_history), return_tensors='pt'), new_input_ids], dim=-1)

        chat_response_ids = model.generate(
            bot_input_ids, 
            max_length=500,
            pad_token_id=tokenizer.eos_token_id,
            top_p=0.95,
            temperature=0.75
        )
        response = tokenizer.decode(chat_response_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

        st.session_state.chat_history.append(f"Bot: {response}")

# Display chat history
for message in st.session_state.chat_history:
    st.write(message)
